# Андреев 6133
# Лабораторная работа №1 по курсу BigData

In [1]:
from pyspark import SparkContext, SparkConf
app_name = "Lab1"
conf = SparkConf().setAppName(app_name).setMaster('local[1]')
sc = SparkContext(conf=conf)
sc

/usr/local/lib/python3.6/dist-packages/pyspark/context.py:238: FutureWarning: Python 3.6 support is deprecated in Spark 3.2.
  FutureWarning


<SparkContext master=local[1] appName=Lab1>

In [2]:
!hadoop fs -put /mnt/data /data

put: `/data/data/list_of_countries_sorted_gini.txt': File exists
put: `/data/data/nycTaxiFares.gz': File exists
put: `/data/data/nycTaxiRides.gz': File exists
put: `/data/data/nyctaxi.txt': File exists
put: `/data/data/posts_sample.xml': File exists
put: `/data/data/programming-languages.csv': File exists
put: `/data/data/stations.csv': File exists
put: `/data/data/trips.csv': File exists
put: `/data/data/warandsociety.txt': File exists


In [3]:
!hadoop fs -ls /data

Found 10 items
drwxr-xr-x   - root root         10 2022-11-24 10:44 /data/data
-rwxr-xr-x   3 root root        394 2022-11-24 05:40 /data/list_of_countries_sorted_gini.txt
-rwxr-xr-x   3 root root   19459967 2022-11-24 05:40 /data/nycTaxiFares.gz
-rwxr-xr-x   3 root root   84135506 2022-11-24 05:40 /data/nycTaxiRides.gz
-rwxr-xr-x   3 root root   79500408 2022-11-24 05:40 /data/nyctaxi.txt
-rwxr-xr-x   3 root root   74162295 2022-11-24 05:40 /data/posts_sample.xml
-rwxr-xr-x   3 root root      40269 2022-11-24 05:40 /data/programming-languages.txt
-rwxr-xr-x   3 root root       5647 2022-11-24 05:40 /data/stations.csv
-rwxr-xr-x   3 root root   80208831 2022-11-24 05:40 /data/trips.csv
-rwxr-xr-x   3 root root    5315699 2022-11-24 05:40 /data/warandsociety.txt


### Tecты

In [4]:
warandpeace = sc.textFile("/mnt/data/warandsociety.txt")
print(warandpeace.count())

12851


In [5]:
nilFile = sc.textFile("nil")

In [6]:
warandpeace.take(10) # Выведем первые 10 элементов

['Лев Николаевич Толстой',
 'Война и мир. Книга 1',
 '',
 'Война и мир – 1',
 '',
 ' ',
 ' http://www.lib.ru',
 '',
 'Аннотация ',
 '']

In [7]:
linesWithPeace = warandpeace.filter(lambda x: "Мир " in x) 
linesWithPeace.first()
# Выведем первую строчку со словом "Мир"

'– Мир заключен… – начал он. Но Наполеон не дал ему говорить. Ему, видно, нужно было говорить самому, одному, и он продолжал говорить с тем красноречием и невоздержанием раздраженности, к которому так склонны балованные люди.'

In [8]:
def time(f):  
    import time
    t = time.process_time()
    f()
    print(f"Elapsed time: {int((time.process_time() - t)*1e9)} ns")

In [9]:
linesWithPeace.cache()
time(lambda: linesWithPeace.count())
time(lambda: linesWithPeace.count())

Elapsed time: 5784800 ns
Elapsed time: 6001799 ns


In [10]:
wordCounts = linesWithPeace\
    .flatMap(lambda line: line.split(" "))\
    .map(lambda word: (word, 1))\
    .reduceByKey(lambda a, b: a + b)
# .flatMap(lambda line: line.split(" "))\   разделим все слова
# .map(lambda word: (word, 1))\   добавим к ним кол-во по умолчанию 1
# .reduceByKey(lambda a, b: a + b)   посчитаем кол-во слов

In [11]:
wordCounts.top(5, lambda x: x[1]) # выведем первые слов с сортировкой по количество слов

[('–', 2), ('говорить', 2), ('и', 2), ('Мир', 1), ('заключен…', 1)]

## Подготовим методы для чтения данных (из вспомогательной методички)

In [12]:
from typing import NamedTuple
from datetime import datetime

def initStation(stations):
    class Station(NamedTuple):
        station_id: int
        name: str
        lat: float
        long: float
        dockcount: int
        landmark: str
        installation: str
    
    for station in stations:
        yield Station(
            station_id = int(station[0]),
            name = station[1],
            lat = float(station[2]),
            long = float(station[3]),
            dockcount = int(station[4]),
            landmark = station[5],
            installation = datetime.strptime(station[6], '%m/%d/%Y')
        )
        
def initTrip(trips):
    class Trip(NamedTuple):
        trip_id: int
        duration: int
        start_date: datetime
        start_station_name: str
        start_station_id: int
        end_date: datetime
        end_station_name: str
        end_station_id: int
        bike_id: int
        subscription_type: str
        zip_code: str
        
    for trip in trips:
        yield Trip(                             
            trip_id = int(trip[0]),
            duration = int(trip[1]) if trip[1] != '' else 0,
            start_date = datetime.strptime(trip[2], '%m/%d/%Y %H:%M') if trip[2] != '' else None,
            start_station_name = trip[3],
            start_station_id = int(trip[4]),
            end_date = datetime.strptime(trip[5], '%m/%d/%Y %H:%M') if trip[5] != '' else None,
            end_station_name = trip[6],
            end_station_id = int(trip[7]),
            bike_id = int(trip[8]),
            subscription_type = trip[9],
            zip_code = trip[10]
        )

In [13]:
def GetDataFromTable(data): # метод, вовзрающий только данные, без названия столбцов
    columns = data.first()
    table = data.filter(lambda row: row != columns)\
                .map(lambda row: row.split(","))
    return columns, table


#### Просто метод, чтобы посмотреть, что все корректно работает

In [14]:
tripData = sc.textFile("/mnt/data/trips.csv")
stationData = sc.textFile("/mnt/data/stations.csv")
print("Trip data\n") 
for data in tripData.take(2):
    print("\n", data)
print("\n\nStation data\n") 
for data in stationData.take(2):
    print("\n", data)

Trip data


 id,duration,start_date,start_station_name,start_station_id,end_date,end_station_name,end_station_id,bike_id,subscription_type,zip_code

 4576,63,,South Van Ness at Market,66,8/29/2013 14:14,South Van Ness at Market,66,520,Subscriber,94127


Station data


 id,name,lat,long,dock_count,city,installation_date

 2,San Jose Diridon Caltrain Station,37.329732,-121.90178200000001,27,San Jose,8/6/2013


In [15]:
tripColumns, trips = GetDataFromTable(tripData)
stationCoolumns, stations = GetDataFromTable(stationData)
stations.first()

['2',
 'San Jose Diridon Caltrain Station',
 '37.329732',
 '-121.90178200000001',
 '27',
 'San Jose',
 '8/6/2013']

In [16]:
!head /mnt/data/stations.csv

id,name,lat,long,dock_count,city,installation_date
2,San Jose Diridon Caltrain Station,37.329732,-121.90178200000001,27,San Jose,8/6/2013
3,San Jose Civic Center,37.330698,-121.888979,15,San Jose,8/5/2013
4,Santa Clara at Almaden,37.333988,-121.894902,11,San Jose,8/6/2013
5,Adobe on Almaden,37.331415,-121.8932,19,San Jose,8/5/2013
6,San Pedro Square,37.336721000000004,-121.894074,15,San Jose,8/7/2013
7,Paseo de San Antonio,37.333798,-121.88694299999999,15,San Jose,8/7/2013
8,San Salvador at 1st,37.330165,-121.88583100000001,15,San Jose,8/5/2013
9,Japantown,37.348742,-121.89471499999999,15,San Jose,8/5/2013
10,San Jose City Hall,37.337391,-121.886995,15,San Jose,8/6/2013


# 1 Найти велосипед с максимальным временем пробега.

In [17]:
tripsObjects = trips.mapPartitions(initTrip)

In [18]:
# Сделаем пары (id байка-его пробег) 
# -> просуммируем все пробеги для каждого байка 
# -> отсортируем полученный массив по duration и выведем первый элемент

print("Id велосипеда с максимальным временем пробега: ",
      tripsObjects.map(lambda trip: (trip.bike_id, trip.duration))\
                  .reduceByKey(lambda a, b: a + b)\
                  .top(1, key = lambda x: x[1])[0][0])

Id велосипеда с максимальным временем пробега:  535


# 2 Найти наибольшее геодезическое расстояние между станциями.

In [19]:
import math
# метод для перевода грудасов в радианы
def degreesToRadians(degrees):
    return math.pi/180*degrees
# метод для расчета расстояния, взятый с сайта stack owerflow
def distanceInKmBetweenEarthCoordinates(lat1, lon1, lat2, lon2):
    earthRadiusKm = 6371

    dLat = degreesToRadians(lat2-lat1)
    dLon = degreesToRadians(lon2-lon1)

    lat1 = degreesToRadians(lat1)
    lat2 = degreesToRadians(lat2)

    a = math.sin(dLat/2) * math.sin(dLat/2) + math.sin(dLon/2) * math.sin(dLon/2) * math.cos(lat1) * math.cos(lat2)
    angle = 2 * math.atan2(math.sqrt(a), math.sqrt(1-a))
    return earthRadiusKm * angle

In [20]:
stationObjects = stations.mapPartitions(initStation)

In [21]:
# Добавим искуственный индекс для соединения таблиц
# -> соединим таблицы
# -> выберем только value в строке (уберем вспомогательный индекс)
# -> для каждой строчки посчитаем расстояние
# -> отсортируем по убыванию
# -> вернем первый элемент
all_id = stationObjects.map(lambda row: (1, (row.station_id, row.lat, row.long)))
answer = all_id.join(all_id)\
               .map(lambda row: row[1])\
      .filter(lambda row: row[0][0]<row[1][0])\
      .map(lambda row: ((row[0][0], row[1][0]),distanceInKmBetweenEarthCoordinates(row[0][1], row[0][2], row[1][1], row[1][2])))\
      .sortBy(lambda x: x[1], ascending=False)\
      .first()
print(answer)
print("Самое большое расстояние между станциями c id=%d и id=%d и равно %f км" %(answer[0][0], answer[0][1],answer[1]))

((16, 60), 69.9208759542826)
Самое большое расстояние между станциями c id=16 и id=60 и равно 69.920876 км


# 3 Найти путь велосипеда с максимальным временем пробега через станции.

In [22]:
# Скопируем запрос из первой задачи и запомним его
idBikeWithMaxDuration = tripsObjects.map(lambda trip: (trip.bike_id, trip.duration))\
            .reduceByKey(lambda a, b: a + b)\
            .top(1, key = lambda x: x[1])[0][0]
# Выберем все поездки на байке с id, у которого максимальный пробег
# -> отсортируем все поездки по дате (по логику где закончил предыдущюю поздке, там же начинается следующая (хотя в теории
# такое не гарантируется в задаче - ремонт при перегонке на другую станцию и т.д.)) 
# -> Оставим только начальные и конечные точки маршрута
allTheWay = tripsObjects.filter(lambda trip: trip.bike_id == idBikeWithMaxDuration)\
                        .sortBy(lambda trip: trip.start_date)\
                        .map(lambda trip: (trip.start_station_name, trip.end_station_name)) 

print("Всего поездок: ", allTheWay.count())
print("\n\n")
print(allTheWay.first()[0], " -> ", end="")
for station in allTheWay.collect():
    print(station[1]," -> ", end="")

Всего поездок:  1328





Post at Kearney  -> San Francisco Caltrain (Townsend at 4th)  -> San Francisco Caltrain 2 (330 Townsend)  -> Market at Sansome  -> 2nd at South Park  -> Davis at Jackson  -> Civic Center BART (7th at Market)  -> Post at Kearney  -> Embarcadero at Sansome  -> Washington at Kearney  -> Market at Sansome  -> Market at Sansome  -> 2nd at Folsom  -> 2nd at Townsend  -> 2nd at Townsend  -> Embarcadero at Sansome  -> Clay at Battery  -> Harry Bridges Plaza (Ferry Building)  -> Clay at Battery  -> San Francisco Caltrain (Townsend at 4th)  -> Steuart at Market  -> 2nd at Townsend  -> Harry Bridges Plaza (Ferry Building)  -> Townsend at 7th  -> San Francisco Caltrain (Townsend at 4th)  -> San Francisco Caltrain 2 (330 Townsend)  -> Townsend at 7th  -> San Francisco Caltrain (Townsend at 4th)  -> 2nd at South Park  -> 5th at Howard  -> San Francisco Caltrain 2 (330 Townsend)  -> Mechanics Plaza (Market at Battery)  -> Powell at Post (Union Square)  -> Powell at Post (Union Square)  -> 5th at Howa

  -> Powell Street BART  -> Embarcadero at Sansome  -> Broadway St at Battery St  -> Mechanics Plaza (Market at Battery)  -> San Francisco Caltrain (Townsend at 4th)  -> South Van Ness at Market  -> 5th at Howard  -> Townsend at 7th  -> Post at Kearny  -> Market at 4th  -> San Francisco Caltrain (Townsend at 4th)  -> Steuart at Market  -> Embarcadero at Sansome  -> 2nd at Townsend  -> Embarcadero at Sansome  -> Steuart at Market  -> San Francisco Caltrain 2 (330 Townsend)  -> Temporary Transbay Terminal (Howard at Beale)  -> San Francisco Caltrain (Townsend at 4th)  -> Market at Sansome  -> South Van Ness at Market  -> San Francisco Caltrain 2 (330 Townsend)  -> Commercial at Montgomery  -> 2nd at Townsend  -> Harry Bridges Plaza (Ferry Building)  -> Embarcadero at Sansome  -> Embarcadero at Sansome  -> Harry Bridges Plaza (Ferry Building)  -> Steuart at Market  -> Embarcadero at Sansome  -> Harry Bridges Plaza (Ferry Building)  -> Yerba Buena Center of the Arts (3rd @ Howard)  -> 2nd 

In [23]:
# Немного полистав данные видим, что не всегда окончание предыдущей поездки есть начало слующей, поэтому
# выведем просто все поездки
for trip in allTheWay.collect():
    print(trip[0], " -> ", trip[1])

Post at Kearney  ->  San Francisco Caltrain (Townsend at 4th)
San Francisco Caltrain (Townsend at 4th)  ->  San Francisco Caltrain 2 (330 Townsend)
San Francisco Caltrain 2 (330 Townsend)  ->  Market at Sansome
Market at Sansome  ->  2nd at South Park
2nd at Townsend  ->  Davis at Jackson
San Francisco City Hall  ->  Civic Center BART (7th at Market)
Civic Center BART (7th at Market)  ->  Post at Kearney
Post at Kearney  ->  Embarcadero at Sansome
Embarcadero at Sansome  ->  Washington at Kearney
Washington at Kearney  ->  Market at Sansome
Market at Sansome  ->  Market at Sansome
Market at Sansome  ->  2nd at Folsom
2nd at Folsom  ->  2nd at Townsend
Temporary Transbay Terminal (Howard at Beale)  ->  2nd at Townsend
2nd at Townsend  ->  Embarcadero at Sansome
Embarcadero at Sansome  ->  Clay at Battery
Clay at Battery  ->  Harry Bridges Plaza (Ferry Building)
Harry Bridges Plaza (Ferry Building)  ->  Clay at Battery
Clay at Battery  ->  San Francisco Caltrain (Townsend at 4th)
San Fra

Howard at 2nd  ->  Embarcadero at Vallejo
Embarcadero at Vallejo  ->  Beale at Market
Beale at Market  ->  Market at 4th
Market at 4th  ->  Harry Bridges Plaza (Ferry Building)
Harry Bridges Plaza (Ferry Building)  ->  Spear at Folsom
Embarcadero at Folsom  ->  Washington at Kearney
Washington at Kearney  ->  Clay at Battery
San Francisco Caltrain (Townsend at 4th)  ->  2nd at Folsom
2nd at Folsom  ->  Clay at Battery
Clay at Battery  ->  5th at Howard
5th at Howard  ->  2nd at Townsend
2nd at Townsend  ->  Market at Sansome
Market at Sansome  ->  Powell at Post (Union Square)
Powell at Post (Union Square)  ->  Embarcadero at Sansome
Embarcadero at Sansome  ->  Powell at Post (Union Square)
Powell at Post (Union Square)  ->  Embarcadero at Sansome
Embarcadero at Sansome  ->  Harry Bridges Plaza (Ferry Building)
Harry Bridges Plaza (Ferry Building)  ->  5th at Howard
5th at Howard  ->  Market at Sansome
Market at Sansome  ->  2nd at South Park
2nd at South Park  ->  San Francisco Caltra

Civic Center BART (7th at Market)  ->  Market at 10th
Market at 10th  ->  Market at 4th
Market at 4th  ->  Embarcadero at Sansome
Embarcadero at Vallejo  ->  Grant Avenue at Columbus Avenue
Powell at Post (Union Square)  ->  San Francisco Caltrain (Townsend at 4th)
San Francisco Caltrain (Townsend at 4th)  ->  Beale at Market
Beale at Market  ->  2nd at South Park
Harry Bridges Plaza (Ferry Building)  ->  Clay at Battery
Clay at Battery  ->  San Francisco Caltrain (Townsend at 4th)
San Francisco Caltrain (Townsend at 4th)  ->  Steuart at Market
Steuart at Market  ->  San Francisco Caltrain 2 (330 Townsend)
Townsend at 7th  ->  Harry Bridges Plaza (Ferry Building)
Harry Bridges Plaza (Ferry Building)  ->  Market at Sansome
Market at Sansome  ->  2nd at Folsom
2nd at Folsom  ->  Davis at Jackson
Davis at Jackson  ->  Clay at Battery
Post at Kearny  ->  South Van Ness at Market
South Van Ness at Market  ->  Temporary Transbay Terminal (Howard at Beale)
2nd at Townsend  ->  Embarcadero at 

Embarcadero at Sansome  ->  Spear at Folsom
Embarcadero at Sansome  ->  Harry Bridges Plaza (Ferry Building)
2nd at Townsend  ->  Davis at Jackson
Spear at Folsom  ->  2nd at Folsom
2nd at Folsom  ->  Spear at Folsom
Spear at Folsom  ->  Grant Avenue at Columbus Avenue
Grant Avenue at Columbus Avenue  ->  Davis at Jackson
Grant Avenue at Columbus Avenue  ->  Commercial at Montgomery
Commercial at Montgomery  ->  San Francisco Caltrain (Townsend at 4th)
2nd at Folsom  ->  Howard at 2nd
Howard at 2nd  ->  Mechanics Plaza (Market at Battery)
Mechanics Plaza (Market at Battery)  ->  Davis at Jackson
Davis at Jackson  ->  Beale at Market
Beale at Market  ->  2nd at Townsend
2nd at Townsend  ->  San Francisco Caltrain (Townsend at 4th)
Market at 10th  ->  Powell Street BART
Powell Street BART  ->  San Francisco Caltrain 2 (330 Townsend)
South Van Ness at Market  ->  2nd at Folsom
2nd at Folsom  ->  Market at Sansome
Market at Sansome  ->  San Francisco Caltrain (Townsend at 4th)
San Francisc

# 4 Найти количество велосипедов в системе.

In [24]:
# Возьмем все id байков -> уберем повторябщиеся -> выведем количество
print("Количество велосипедов:", tripsObjects.map(lambda trip: trip.bike_id).distinct().count())

Количество велосипедов: 700


# 5 Найти пользователей потративших на поездки более 3 часов.

In [25]:
# Три часа
threeHours = 3 * 60 * 60
# Выбираем поездки, дольше трех часов ->  оставляем только id
query = tripsObjects.filter(lambda trip: trip.duration > threeHours).map(lambda trip: trip.trip_id)
print("Количество: ", query.count())
print(query.collect())

Количество:  8322
[4639, 4637, 4528, 4363, 4193, 4190, 4225, 4663, 4532, 4521, 5069, 4505, 5539, 6032, 6409, 6408, 5697, 5683, 5218, 5716, 5717, 5241, 5537, 5253, 5531, 6131, 6098, 6091, 5115, 5114, 5250, 5249, 5508, 5506, 5167, 5166, 5640, 5638, 6003, 5380, 5222, 5120, 5113, 5182, 6412, 6178, 6462, 5493, 6282, 7069, 6626, 6963, 7074, 6962, 7067, 6792, 7143, 6613, 7176, 7081, 7079, 7021, 7016, 6877, 6876, 6828, 7110, 7106, 6806, 6821, 7005, 6756, 7085, 7084, 6748, 6654, 6741, 7015, 6798, 6796, 6769, 6771, 6968, 6966, 6752, 6745, 6867, 6864, 7182, 7179, 6906, 7178, 6824, 6679, 6656, 6822, 7434, 7437, 7424, 7180, 6633, 6631, 7208, 7044, 7466, 7495, 7497, 7499, 7501, 7503, 7521, 7529, 7550, 7551, 7553, 7557, 7559, 7569, 7579, 7624, 7625, 7630, 7639, 7645, 7647, 7655, 7686, 7692, 7735, 7767, 7774, 7799, 7805, 7832, 7849, 7851, 7862, 7929, 7931, 7948, 7956, 8085, 8197, 8204, 8373, 8391, 8438, 8445, 8446, 8532, 8583, 8587, 8589, 8591, 8624, 8627, 8740, 8755, 8813, 8821, 8830, 8839, 8842, 884